In [12]:
# Web_API_Extraction from Twitter
# This class containes the code for extracting popularity information related to cities through Twitter API.
# Detailed description availble through inline comments.

import twitter# pip install twitter
import json 
import pandas as pd
from urllib.parse import unquote
import time



# Storing relevant keys and token information in desired variables
consumer_key = "8kN7ITYYYK6bNkHKoViRPb7gJ"
consumer_secret = "g8IGSzKWckrYRzYjTgm8J2oNIywsmzcS8I9zW6WTrtIx28RGrE"
access_token = "1088871960698908674-WsDWHHSB2jcaH9zMwaW5mNgpwIzpGM"
access_token_secret = "0pHfCkkkjEhWeQbm6tZ9eWHRLXzIh4dyNimAHXmuvYg6j"



# Initialising twitter api
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)



# Reading the data extracted from kaggel, which contains the list of players, whose popularity information is what is 
# going to be extracted from twitter 
cities_input = pd.read_csv('City.csv')
#Constructing a dictionary from two columns of the read data namely 'Player_ID' and 'Player_Name' adn stored in cricket_players_dict
cities_dict= dict(zip(cities_input.city_id, cities_input.city_name))

# Initialising below empty lists for storing player popularity related information
city_id_list = []
tweet_content_list = []
tweet_datetime_list = []
favourite_count_list = []
retweet_count_list = []
user_id_list = []
tweet_id_list = []
user_name_list = []
user_screen_name_list = []
user_followers_count_list = []
hashtag_tweet_id_list = []
hashtag_content_list = []
hashtag_id_list = []


#Iterating through the items of dictionary 'cities_dict'
for key, value in cities_dict.items():

        
    # Searching using city name as inout
    results = api.GetSearch(raw_query="q=" +value+ "&count=5")
    cnt = len(results)
    
    
    #Iterating through the results
    for res in results:
        x = json.loads(str(res))
        hashtags_used = []
        
        # Considering only tweets in English Language
        if x['lang']=='en':      
            # Storing the 'key'(cityid) in city_id_list
            city_id_list.append(key)
            # Storing the actual text of the tweet in tweet_content_list
            tweet_content_list.append(x['text'])
            # Converting the date into readable formate and storing in tweet_datetime_list
            tweet_datetime_list.append(time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(x['created_at'],'%a %b %d %H:%M:%S +0000 %Y')))
            favourite_count = 0
            # IF there is an exception while accessing 'Favorite_count', then 'favorite_count' from 'retweeted_status' is used
            try:
                try:
                    favourite_count = x['favorite_count']
                except:
                    favourite_count = x['retweeted_status']['favorite_count']
            except:
                 favourite_count = 0   
            favourite_count_list.append(favourite_count)
            
            # If there is an exception with accesing 'retweet_count', then 0 is stored
            try:
                retweet_count_list.append(x['retweet_count'])
            except:
                retweet_count_list.append(0)
                
            # String version of userid and tweetid are stored in user_id_list and tweet_id_list
            user_id_list.append(x['user']['id_str'])
            tweet_id_list.append(x['id_str'])
            
            # 'User name' and 'User Screen name' are stored in user_name_list and user_screen_name_list respectively
            user_name_list.append(x['user']['name'])
            user_screen_name_list.append(x['user']['screen_name'])
            
            # If there is an exception with accessing user's followers_count, then 0 is stored
            try:
                user_followers_count_list.append(x['user']['followers_count'])
            except:
                user_followers_count_list.append(0)
            
            # If there is an exception with accessing first hashtag's text, then entire information from hashtags are stored
            try:
                hashtags_used.append(x['hashtags'][0]['text'])
            except:
                hashtags_used.append(x['hashtags'])
            
           # Empty lists which were stored in 'hashtags_used' list are removed and the corrected version is stored in 
           # hashtags_used_corrected
            hashtags_used_empty_lists_removed = [x for x in hashtags_used if x != []]
            hashtags_used_corrected = [x for x in hashtags_used_empty_lists_removed if x]    
            
            #Iterating through list of all retrieved and stored hashtags
            for hashtag in hashtags_used_corrected:
                
                # Storign the respective hashtag information in relevant lists 
                # Storing the string version of hashtag tweed id in hashtag_tweet_id_list
                hashtag_tweet_id_list.append(x['id_str'])
                hashtag_content_list.append(hashtag)
                # Hashtag id is created by concatenating hahstag id and string version of the entire hashtag
                hashtag_id_list.append(x['id_str'] + str(hashtag))
                
                
                
# Creating a dataframe city_tweet_df to store tweet information collected with regard to cities         
city_tweet_df = pd.DataFrame(data = city_id_list, columns = ['city_id'])
city_tweet_df['Content'] = tweet_content_list  
city_tweet_df['Created_Time'] = tweet_datetime_list
city_tweet_df['Favourite_Count'] = favourite_count_list
city_tweet_df['Retweet_Count'] = retweet_count_list
city_tweet_df['Tweet_ID'] = tweet_id_list
city_tweet_df['User_ID'] = user_id_list


# Rename column 'city_id' to 'City_ID'
city_tweet_df = city_tweet_df.rename(columns={'city_id': 'City_ID'})


# Creating a dataframe city_twitter_user_df to store information about user, who had posted tweets collected with regard to cities           
city_twitter_user_df = pd.DataFrame(data = user_id_list , columns = ['User_ID'])
city_twitter_user_df['User_Name'] = user_name_list
city_twitter_user_df['User_Screen_Name'] = user_screen_name_list
city_twitter_user_df['Followers_Count'] = user_followers_count_list


# Creating a dataframe city_twitter_hashtag_df  to store information about hashtags which were used in tweets collected with regard to cities           
city_twitter_hashtag_df = pd.DataFrame(data = hashtag_tweet_id_list , columns = ['Tweet_ID'])
city_twitter_hashtag_df ['Name'] = hashtag_content_list
city_twitter_hashtag_df ['Hashtag_ID'] = hashtag_id_list



# Removal of duplicate rows with regard to 'Tweet_ID', 'User_ID' and 'Hashtag_ID' fields in the below dataframes 
city_tweet_df.drop_duplicates(subset=['Tweet_ID'], keep='first', inplace= True)
city_twitter_user_df.drop_duplicates(subset=['User_ID'], keep='first', inplace= True)
city_twitter_hashtag_df.drop_duplicates(subset=['Hashtag_ID'], keep='first', inplace= True)

# Exporting dataframes to respective csv files
city_tweet_df.to_csv("City_Twitter_Extracted_Tweet.csv", index=False)
city_twitter_user_df.to_csv("City_Twitter_Extracted_TwitterUser.csv",index=False)
city_twitter_hashtag_df.to_csv("City_Twitter_Extracted_TwitterHashtag.csv", index=False)



""""
CONCLUSIONS:

This code uses the list of cities, selected for the project and  tries to find the popularity parameters related 
to the same through Twitter API, stores the results in dataframe and exports it to a CSV file.

CONTRIBUTIONS:

RAJENDRA KUMAR RAJKUMAR - 75% 
MONISH  HIRISAVE RAGHU - 25%

CITATIONS:

1. https://www.geeksforgeeks.org
2. https://github.com/nikbearbrown/INFO_6210

The code with regard to extraction of information from twitter was used from the above mentioned resources

Original writtten code - 60%
Code referenced from external sources(but modified suiting needs) - 40%

LICENSE:

Copyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


"""

'"\nCONCLUSIONS:\n\nThis code uses the list of cities, selected for the project and  tries to find the popularity parameters related \nto the same through Twitter API, stores the results in dataframe and exports it to a CSV file.\n\nCONTRIBUTIONS:\n\nRAJENDRA KUMAR RAJKUMAR - 75% \nMONISH  HIRISAVE RAGHU - 25%\n\nCITATIONS:\n\n1. https://www.geeksforgeeks.org\n2. https://github.com/nikbearbrown/INFO_6210\n\nThe code with regard to extraction of information from twitter was used from the above mentioned resources\n\nOriginal writtten code - 60%\nCode referenced from external sources(but modified suiting needs) - 40%\n\nLICENSE:\n\nCopyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>\n\nPermission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense

In [10]:
#results = api.trends.place(_id = 23424975)

#print("UK Trends")

#for location in results:
#    for trend in location["trends"]:
#        print(" - %s" % trend["name"])
        
print(api.)

AttributeError: 'Api' object has no attribute 'trends'